In [ ]:
# About the dataset 
# The dataset contains the following columns

# Port_IDs: Unique identifiers for binance accounts.
# Trade_History: Historical trade data stored as a JSON-like string for each account.
# Key observations

# There are 150 rows, with one missing value in the Trade_History column.
# The Trade_History column contains nested data in JSON format.


import pandas as pd
import ast
import numpy as np


file_path = '/Users/harshrudani/Desktop/Online Assesement/TRADES_CopyTr_90D_ROI.csv'  # Update the file path
trade_data = pd.read_csv(file_path)

# Dropping rows with missing Trade_History (1 row with missing value)
trade_data = trade_data.dropna(subset=['Trade_History'])

# Parsing the Trade_History column
trade_data['Trade_History'] = trade_data['Trade_History'].apply(ast.literal_eval)

# Function to calculate metrics for a single account
def calculate_metrics(trade_history):
    try:
       
        df = pd.DataFrame(trade_history)
        
        # Ensuring numeric columns and also handling missing values by filling with column mean
        for col in ['realizedProfit', 'quantity']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')  # Ensure numeric
                mean_value = df[col].mean() if not df[col].isnull().all() else 0
                df[col] = df[col].fillna(mean_value)  # Fill null values with mean

        # Fill missing or unknown values
        df = df.fillna({'side': 'UNKNOWN', 'positionSide': 'UNKNOWN'})

        # Combining side and positionSide to classify trades
        df['trade_type'] = df['side'] + '_' + df['positionSide']
        
        # Calculating ROI
        total_investment = df.loc[df['side'] == 'BUY', 'quantity'].sum()
        total_profit = df['realizedProfit'].sum()
        roi = (total_profit / total_investment) if total_investment > 0 else 0
        
        # Calculating Sharpe Ratio
        daily_returns = df['realizedProfit']
        sharpe_ratio = (daily_returns.mean() / daily_returns.std()) * np.sqrt(252) if daily_returns.std() > 0 else 0
        
        # Calculating Maximum Drawdown 
        cumulative_returns = (1 + daily_returns).cumprod()
        peak = cumulative_returns.cummax()
        drawdown = (cumulative_returns - peak) / peak
        mdd = drawdown.min() if not drawdown.empty else 0
        
        # Calculating Win Rate and Win Positions
        wins = (df['realizedProfit'] > 0).sum()
        total_positions = len(df)
        win_rate = (wins / total_positions) if total_positions > 0 else 0
        
        return roi, total_profit, sharpe_ratio, mdd, win_rate, wins, total_positions, df['trade_type'].unique()
    except Exception as e:
        print(f"Error processing trade history: {e}")
        return 0, 0, 0, 0, 0, 0, 0, []

# Calculating  metrics for all accounts
metrics = []
for _, row in trade_data.iterrows():
    port_id = row['Port_IDs']
    roi, pnl, sharpe, mdd, win_rate, wins, total_positions, trade_types = calculate_metrics(row['Trade_History'])
    metrics.append({
        'Port_IDs': port_id,
        'ROI': roi,
        'PnL': pnl,
        'Sharpe_Ratio': sharpe,
        'MDD': mdd,
        'Win_Rate': win_rate,
        'Win_Positions': wins,
        'Total_Positions': total_positions,
        'Trade_Types': ', '.join(trade_types)  # Combining trade types into  string
    })

# dataframe for metrics
metrics_df = pd.DataFrame(metrics)

# Ranking account based on metrics
metrics_df['Rank'] = metrics_df[['ROI', 'PnL', 'Sharpe_Ratio', 'Win_Rate']].rank(
    method='min', ascending=False).mean(axis=1)

# Selecting the top 20 account based on rank
top_20_accounts = metrics_df.sort_values(by='Rank', ascending=True).head(20)

# Save results to CSV
metrics_df.to_csv('calculated_metrics.csv', index=False)
top_20_accounts.to_csv('top_20_accounts.csv', index=False)

# Summary
print("Metrics calculated and saved to 'calculated_metrics.csv'")
print("Top 20 accounts saved to 'top_20_accounts.csv'")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)
/Library/Frameworks/Python.framework/Versions

Metrics calculated and saved to 'calculated_metrics.csv'
Top 20 accounts saved to 'top_20_accounts.csv'


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)
/Library/Frameworks/Python.framework/Versions